## Requirements

In [1]:
import numpy as np
from deep_teaching_commons.data.fundamentals.mnist import Mnist

import os
import pickle

# used to check whether datasets need to be downloaded
from pathlib import Path

## Data

In [2]:
def load_CIFAR_batch(filename):  
    with open(filename, 'rb') as f:      
        datadict = pickle.load(f, encoding='bytes')
        X = datadict[b'data']
        Y = datadict[b'labels']
        # Create a numpy array with dimensions (n, channel, height, width)
        X = X.reshape(10000, 3, 32, 32).astype("float64")
        # Squash all values into interval [0,1]
        X = X / 256.
        Y = np.array(Y).astype("uint8")
        return X, Y

def load_CIFAR10(root_folder):
    
    Xtr = np.zeros((50000,3,32,32))
    Ytr = np.zeros((50000)).astype("uint8")
    batch_size = 10000
    
    # Load and decode each batch of the trainings set
    for index, batch in enumerate(range(1,6)):
        f = os.path.join(root_folder, 'data_batch_%d' % batch)
        print(f)
        
        # Boundary indices of the current batch within the overall trainings set
        b_i_start = index * batch_size
        b_i_end = b_i_start + batch_size
        
        # Load and decode trainigs data and labels
        Xtr[b_i_start:b_i_end], Ytr[b_i_start:b_i_end] = load_CIFAR_batch(f)

    # Load and decode test data and labels  
    Xte, Yte = load_CIFAR_batch(os.path.join(root_folder, 'test_batch'))
    
    return Xtr, Ytr, Xte, Yte

cifar10_dir = 'cifar-10-batches-py'  # default dir

if not Path('cifar-10-batches-py').is_dir():
    print('Downloading data sets...')
    !sh get_datasets.sh
else:
    print('Data sets already downloaded!')

train_images, train_labels, test_images, test_labels = load_CIFAR10(cifar10_dir)

Data sets already downloaded!
cifar-10-batches-py/data_batch_1
cifar-10-batches-py/data_batch_2
cifar-10-batches-py/data_batch_3
cifar-10-batches-py/data_batch_4
cifar-10-batches-py/data_batch_5


## Layers

In [3]:
class Flatten():
    ''' Flatten layer used to reshape inputs into vector representation
    
    Layer should be used in the forward pass before a dense layer to 
    transform a given tensor into a vector. 
    '''
    def __init__(self):
        self.params = []

    def forward(self, X):
        ''' Reshapes a n-dim representation into a vector 
            by preserving the number of input rows.
        
        Examples:
            [10000,[1,28,28]] -> [10000,784]
        '''
        self.X_shape = X.shape
        self.out_shape = (self.X_shape[0], -1)    
        out = X.reshape(-1).reshape(self.out_shape)
        return out

    def backward(self, dout):
        ''' Restore dimensions before flattening operation
        '''
        out = dout.reshape(self.X_shape)
        return out, []

class FullyConnected():
    ''' Fully connected layer implemtenting linear function hypothesis 
        in the forward pass and its derivation in the backward pass.
    '''
    def __init__(self, in_size, out_size):
        ''' Initilize all learning parameters in the layer
        
        Weights will be initilized with modified Xavier initialization.
        Biases will be initilized with zero. 
        '''
        self.W = np.random.randn(in_size, out_size) * np.sqrt(2. / in_size)
        self.b = np.zeros((1, out_size))
        self.params = [self.W, self.b]

    def forward(self, X):
        self.X = X
        out = np.add(np.dot(self.X, self.W), self.b)
        return out

    def backward(self, dout):
        dX = np.dot(dout, self.W.T)
        dW = np.dot(self.X.T, dout)
        db = np.sum(dout, axis=0)
        return dX, [dW, db]

## Activation Function

In [4]:
class ReLU():
    ''' Implements activation function rectified linear unit (ReLU) 
    
    ReLU activation function is defined as the positive part of 
    its argument. Todo: insert arxiv paper reference
    '''
    def __init__(self):
        self.params = []

    def forward(self, X):
        ''' In the forward pass return the identity for x < 0
        
        Safe input for backprop and forward all values that are above 0.
        '''
        self.X = X
        return np.maximum(X, 0)

    def backward(self, dout):
        ''' Derivative of ReLU
        
        Retruns:
            dX: for all x \elem X <= 0 in forward pass 
                return 0 else x
            []: no gradients on ReLU operation
        '''
        dX = dout.copy()
        dX[self.X <= 0] = 0
        return dX, []

## Neural Network class

In [5]:
class NeuralNetwork:
    ''' Creates a neural network from a given layer architecture 
    
    This class is suited for fully connected network and
    convolutional neural network architectures. It connects 
    the layers and passes the data from one end to another.
    '''
    def __init__(self, layers, score_func=None):
        ''' Setup a global parameter list and initilize a
            score function that is used for predictions.
        
        Args:
            layer: neural network architecture based on layer and activation function objects
            score_func: function that is used as classifier on the output
        '''
        self.layers = layers
        self.params = []
        for layer in self.layers:
            self.params.append(layer.params)
        self.score_func = score_func

    def forward(self, X):
        ''' Pass input X through all layers in the network 
        '''
        for layer in self.layers:
            X = layer.forward(X)
        return X

    def backward(self, dout):
        grads = []
        ''' Backprop through the network and keep a list of the gradients
            from each layer.
        '''
        for layer in reversed(self.layers):
            dout, grad = layer.backward(dout)
            grads.append(grad)
        return grads

    def predict(self, X):
        ''' Run a forward pass and use the score function to classify 
            the output.
        '''
        X = self.forward(X)
        return np.argmax(self.score_func(X), axis=1)

## Loss function

In [6]:
class LossCriteria():
    ''' Implements diffrent typs of loss and score functions for neural networks
    
    Todo:
        - Implement init that defines score and loss function 
    '''
    def softmax(X):
        ''' Numeric stable calculation of softmax
        '''
        exp_X = np.exp(X - np.max(X, axis=1, keepdims=True))
        return exp_X / np.sum(exp_X, axis=1, keepdims=True)

    def cross_entropy_softmax(X, y):
        ''' Computes loss and prepares dout for backprop 

        https://eli.thegreenplace.net/2016/the-softmax-function-and-its-derivative/
        '''
        m = y.shape[0]
        p = LossCriteria.softmax(X)
        log_likelihood = -np.log(p[range(m), y])
        loss = np.sum(log_likelihood) / m
        dout = p.copy()
        dout[range(m), y] -= 1
        return loss, dout

## Optimization with SGD

In [7]:
class Optimizer():   
    def get_minibatches(X, y, batch_size):
        ''' Decomposes data set into small subsets (batch)
        '''
        m = X.shape[0]
        batches = []
        for i in range(0, m, batch_size):
            X_batch = X[i:i + batch_size, :, :, :]
            y_batch = y[i:i + batch_size, ]
            batches.append((X_batch, y_batch))
        return batches    

    def sgd(network, X_train, y_train, loss_function, batch_size=32, epoch=100, learning_rate=0.001, X_test=None, y_test=None, verbose=None):
        ''' Optimize a given network with stochastic gradient descent 
        '''
        minibatches = Optimizer.get_minibatches(X_train, y_train, batch_size)
        for i in range(epoch):
            loss = 0
            if verbose:
                print('Epoch',i + 1)
            for X_mini, y_mini in minibatches:
                # calculate loss and derivation of the last layer
                loss, dout = loss_function(network.forward(X_mini), y_mini)
                # calculate gradients via backpropagation
                grads = network.backward(dout)
                # run vanilla sgd update for all learnable parameters in self.params
                for param, grad in zip(network.params, reversed(grads)):
                    for i in range(len(grad)):
                        param[i] += - learning_rate * grad[i]
            if verbose:
                train_acc = np.mean(y_train == network.predict(X_train))
                test_acc = np.mean(y_test == network.predict(X_test))                                
                print("Loss = {0} :: Training = {1} :: Test = {2}".format(loss, train_acc, test_acc))
        return network

In [8]:
# shuffle training data
shuffle_index = np.random.permutation(len(train_images))
train_images, train_labels = train_images[shuffle_index], train_labels[shuffle_index]

train_images_batch, train_labels_batch, test_images_batch, test_labels_batch = train_images[:5000,:,:,:], train_labels[:5000], test_images[:1000,:,:,:], test_labels[:1000]

## Output

In [ ]:
# design a three hidden layer architecture with Dense-Layer
# and ReLU as activation function
def fcn_mnist():
    flat = Flatten()
    hidden_01 = FullyConnected(3072, 500)
    relu_01 = ReLU()
    hidden_02 = FullyConnected(500, 200)
    relu_02 = ReLU()
    hidden_03 = FullyConnected(200, 100)
    relu_03 = ReLU()
    ouput = FullyConnected(100, 10)
    return [flat, hidden_01, relu_01, hidden_02, relu_02, hidden_03, relu_03, ouput]

# create a neural network on specified architecture with softmax as score function
fcn = NeuralNetwork(fcn_mnist(), score_func=LossCriteria.softmax)

# optimize the network and a softmax loss
fcn = Optimizer.sgd(fcn, train_images_batch, train_labels_batch, LossCriteria.cross_entropy_softmax, batch_size=64, epoch=30, learning_rate=0.0001, X_test=test_images_batch, y_test=test_labels_batch, verbose=True)

Epoch 1
Loss = 2.388660988323428 :: Training = 0.2688 :: Test = 0.233
Epoch 2
Loss = 2.3497366148629206 :: Training = 0.3064 :: Test = 0.273
Epoch 3
Loss = 2.31800625088761 :: Training = 0.3314 :: Test = 0.292
Epoch 4
Loss = 2.2769367338409774 :: Training = 0.3486 :: Test = 0.29
Epoch 5
Loss = 2.2253004058344 :: Training = 0.3568 :: Test = 0.304
Epoch 6
Loss = 2.1772647447003353 :: Training = 0.3692 :: Test = 0.309
Epoch 7
Loss = 2.1360943895689344 :: Training = 0.3838 :: Test = 0.315
Epoch 8
Loss = 2.1168465140266797 :: Training = 0.3958 :: Test = 0.327
Epoch 9
Loss = 2.0908992319939372 :: Training = 0.4092 :: Test = 0.33
Epoch 10
Loss = 2.055116970279644 :: Training = 0.4158 :: Test = 0.331
Epoch 11
Loss = 2.0283905549952834 :: Training = 0.426 :: Test = 0.338
Epoch 12
Loss = 1.994532378301266 :: Training = 0.4304 :: Test = 0.338
Epoch 13
Loss = 1.9744294503278574 :: Training = 0.4382 :: Test = 0.351
Epoch 14
Loss = 1.9627331961061658 :: Training = 0.4472 :: Test = 0.356
Epoch 15
Los